In [ ]:
# TODO : converting using numpy take really long time, find another way to do the conversion 

In [12]:
import os 
import numpy as np
import cv2


labels = [
    (  'void'  , (  0,  0,  0) ), 
    (  'void'  , (111, 74,  0) ),
    (  'void'  , ( 81,  0, 81) ),
    (  'flat'  , (128, 64,128) ),
    (  'flat'  , (244, 35,232) ),
    (  'flat'  , (250,170,160) ),
    (  'flat'  , (230,150,140) ),
    (  'construction'  , ( 70, 70, 70) ),
    (  'construction'  , (102,102,156) ),
    (  'construction'  , (190,153,153) ),
    (  'construction'  , (180,165,180) ),
    (  'construction'  , (150,100,100) ),
    (  'construction'  , (150,120, 90) ),
    (  'object'    , (153,153,153) ),
    (  'object'    , (250,170, 30) ),
    (  'object'    , (220,220,  0) ),
    (  'nature'    , (107,142, 35) ),
    (  'nature'    , (152,251,152) ),
    (  'sky'       , ( 70,130,180) ),
    (  'human'     , (220, 20, 60) ),
    (  'human'     , (255,  0,  0) ),
    (  'vehicle'   , (  0,  0,142) ),
    (  'vehicle'   , (  0,  0, 70) ),
    (  'vehicle'   , (  0, 60,100) ),
    (  'vehicle'   , (  0,  0, 90) ),
    (  'vehicle'   , (  0,  0,110) ),
    (  'vehicle'   , (  0, 80,100) ),
    (  'vehicle'   , (  0,  0,230) ),
    (  'vehicle'   , (119, 11, 32) )]

category=dict([('void',0),('flat',1),('construction',2),('object',3),('nature',4),('sky',5),('human',6),('vehicle',7)])

def fill_in_empty(mask,placeholder,tp,current_label):
    for i in range(1024):
        for j in range(2048):
            if tp[i,j]:
                placeholder[i,j,0]=current_label                
    return placeholder


def re_create_mask(mask, placeholder, label_name, label_color, current_label, current_color, category):
    if current_label==label_name and current_color==label_color:
        #placeholder=np.zero(shape=(1024,2048,1), dtype=np.float)
        tp=np.all(mask==current_color,axis=-1)
        placeholder=fill_in_empty(mask,placeholder,tp,category[current_label])
        return placeholder , current_label, current_color
    elif current_label==label_name and current_color!=label_color:
        current_color=label_color
        tp=np.all(mask==current_color,axis=-1)
        placeholder=fill_in_empty(mask,placeholder,tp,category[current_label])
        return placeholder, current_label, current_color
    elif current_label!=label_name and current_color!=label_color:
        current_label=label_name
        current_color=label_color
        tp=np.all(mask==current_color,axis=-1)
        placeholder=fill_in_empty(mask,placeholder,tp,category[current_label])
        return placeholder, current_label, current_color
    else:
        print("something is wrong, should have already eliminate same color, different label problems")
        
show=3
i=0
show_labels=False
visualize=True


def outer_recreate(ma, labels):
    idx=0
    placeholder=np.zeros(shape=(1024,2048,1), dtype=np.float)
    for  label_name, label_color in labels:
        if idx==0:
            current_label=label_name
            current_color=label_color
            placeholder, current_label, current_color=re_create_mask(ma, placeholder, label_name, label_color, current_label, current_color, category)
            idx+=1
        else :
            placeholder, current_label, current_color=re_create_mask(ma, placeholder, label_name, label_color, current_label, current_color, category)
            idx+=1
    return placeholder
gt_path='/workspace/8data/gt/'
out_path='/workspace/8data/gt_cat/'
gt_files=os.listdir(gt_path)
##### TODO : convert pre-processing step from numpy to more efficient library, it took too long to convert to categorical label masks as images
########### do not run the below prep step using numpy it took too long 
"""
for gt_file in gt_files:
    
    file_name= gt_file.split('.')[0]+'.png'
    #print(out_path+file_name, placeholder.shape)
    #np.save(out_path+file_name, placeholder)
    if os.path.exists(out_path+file_name):
        pass
    else:
        ma=cv2.imread(gt_path+gt_file)
        ma=ma[:,:,[2,1,0]]
        placeholder=outer_recreate(ma, labels)
        cv2.imwrite(out_path+file_name,placeholder)
print("saving complete for {} images".format(str(len(gt_files))))"""

saving complete for 2975 images


In [1]:
import os
imP='./raw/leftImg8bit/train/'
maP='./raw/gtFine/train/'
imO='/workspace/8data/img/'
maO='/workspace/8data/gt_cat/'

len(os.listdir(imO)),len(os.listdir(maO))

    

(2975, 2975)

In [8]:
ma=cv2.imread(gt_path+gt_files[0])
ma=ma[:,:,[2,1,0]]
placeholder=outer_recreate(ma, labels)
file_name= gt_files[0].split('.')[0]+'.png'
print(out_path+file_name, placeholder.shape)
#np.save(out_path+file_name, placeholder)
cv2.imwrite(out_path+file_name,placeholder)
import random
rni=[random.randint(0,1023) for _ in range(10)]
rnj=[ random.randint(0,2047) for _ in range(10)]
print(placeholder.shape,placeholder[rni,rnj,:])
new=cv2.imread(out_path+file_name,cv2.IMREAD_UNCHANGED)

print(new.shape,new[rni,rnj])

/workspace/hiRes_Unet/8data/gt_cat/weimar_000136_000019_gtFine_color.png (1024, 2048, 1)
(1024, 2048, 1) [[4.]
 [2.]
 [0.]
 [7.]
 [0.]
 [4.]
 [1.]
 [4.]
 [1.]
 [1.]]
(1024, 2048) [4 2 0 7 0 4 1 4 1 1]


In [4]:
import os
import numpy as np
imO='/workspace/8data/img/'
maO='/workspace/8data/gt_cat/'
im_ls=os.listdir(imO)
ma_ls=os.listdir(maO)

with open("img8_small.txt", "w") as f1:
    with open("gt_cat8_small.txt", "w") as f2:
        
        for im_name in im_ls:
            keys=im_name.split('.')[0].split('_')[:3]
            mask_name='_'.join(keys)+'_gtFine_color'+'.png' 
            
            if os.path.exists(maO+mask_name):
                #print(im_name,mask_name)
                f1.write('%-1s\n' % (imO+im_name))
                #f2.write('%-1s,%s\n' % (imO+im_name,maO+mask_name))
                f2.write('%-1s\n' % (maO+mask_name))  



In [2]:
!head img8_small.txt

/workspace/8data/img/strasbourg_000000_034387_leftImg8bit.png
/workspace/8data/img/ulm_000040_000019_leftImg8bit.png
/workspace/8data/img/hanover_000000_052729_leftImg8bit.png
/workspace/8data/img/hanover_000000_048379_leftImg8bit.png
/workspace/8data/img/stuttgart_000019_000019_leftImg8bit.png
/workspace/8data/img/darmstadt_000008_000019_leftImg8bit.png
/workspace/8data/img/hamburg_000000_098616_leftImg8bit.png
/workspace/8data/img/krefeld_000000_029704_leftImg8bit.png
/workspace/8data/img/strasbourg_000001_058373_leftImg8bit.png
/workspace/8data/img/ulm_000043_000019_leftImg8bit.png


In [5]:
!head gt_cat8_small.txt

/workspace/8data/gt_cat/strasbourg_000000_034387_gtFine_color.png
/workspace/8data/gt_cat/ulm_000040_000019_gtFine_color.png
/workspace/8data/gt_cat/hanover_000000_052729_gtFine_color.png
/workspace/8data/gt_cat/hanover_000000_048379_gtFine_color.png
/workspace/8data/gt_cat/stuttgart_000019_000019_gtFine_color.png
/workspace/8data/gt_cat/darmstadt_000008_000019_gtFine_color.png
/workspace/8data/gt_cat/hamburg_000000_098616_gtFine_color.png
/workspace/8data/gt_cat/krefeld_000000_029704_gtFine_color.png
/workspace/8data/gt_cat/strasbourg_000001_058373_gtFine_color.png
/workspace/8data/gt_cat/ulm_000043_000019_gtFine_color.png
